In [1]:
import os
import pandas as pd
import nltk
nltk.download('punkt')
import gensim
import numpy as np
from gensim import corpora, models, similarities
from gensim.models import Word2Vec
import logging
import gzip

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rizary\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


C:\Users\Rizary\AppData\Roaming\Python\Python36\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df=pd.read_json('./file.json')

In [3]:
x = df['title'].values.tolist()
y = df['content'].values.tolist()

In [4]:
corpus = x+y

In [5]:
tok_corp = [nltk.word_tokenize(sent) for sent in corpus]

In [6]:
model = Word2Vec(tok_corp,min_count=1,size=32)

In [7]:
w1 = "the"
model.wv.most_similar(positive=w1)

[('to', 0.9998850226402283),
 ('.', 0.9998338222503662),
 (',', 0.9998176693916321),
 ('and', 0.9997790455818176),
 ('has', 0.9997731447219849),
 ("'s", 0.9997619390487671),
 ('a', 0.9997375011444092),
 ('that', 0.9997373819351196),
 ('of', 0.9997318387031555),
 ('was', 0.999695897102356)]

In [11]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

In [29]:
def visualize(model, output_path):
    meta_file = "w2x_metadata.tsv"
    placeholder = np.zeros((len(model.wv.index2word), 32))

    with open(os.path.join(output_path,meta_file), 'wb') as file_metadata:
        for i, word in enumerate(model.wv.index2word):
            placeholder[i] = model[word]
            # temporary solution for https://github.com/tensorflow/tensorflow/issues/9094
            if word == '':
                print("Emply Line, should replecaed by any thing else, or will cause a bug of tensorboard")
                file_metadata.write("{0}".format('<Empty Line>').encode('utf-8') + b'\n')
            else:
                file_metadata.write("{0}".format(word).encode('utf-8') + b'\n')

    # define the model without training
    sess = tf.InteractiveSession()

    embedding = tf.Variable(placeholder, trainable = False, name = 'w2x_metadata')
    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(output_path, sess.graph)

    # adding into projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = 'w2x_metadata'
    embed.metadata_path = meta_file

    # Specify the width and height of a single thumbnail.
    projector.visualize_embeddings(writer, config)
    saver.save(sess, os.path.join(output_path,'w2x_metadata.ckpt'))
    
    print('Run `tensorboard --logdir={0}` to run visualize result on tensorboard'.format(output_path))

In [30]:
model1=Word2Vec.load('./model_cnn_edition')

In [31]:
visualize(model1,"./logs")

d:\engineer\ide\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
d:\engineer\ide\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Run `tensorboard --logdir=./logs` to run visualize result on tensorboard


In [14]:

model.save('model_cnn_edition')